In [1]:
from bs4 import BeautifulSoup as bs
import requests
import redis
import time
import numpy as np
import lda
import lda.datasets
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
r = redis.StrictRedis()
swfa = pd.read_csv('../dataset/stopwords.csv' , header=None)

NameError: name 'redis' is not defined

In [ ]:
start_time = time.time()
for i in range(82120000,82154440):#range(82120000,82154440):
    if i % 500 == 0:
        print(i)
    if r.exists('irna#'+str(i)):
        continue
    url = 'http://www.irna.ir/fa/News/'+str(i)+'/'
    page = requests.get(url).text
    soup = bs(page , "html.parser")
    if soup :
        body = soup.find('div',class_='BodyText')
        if body:
            body = body.find('p')
            r.set('irna#'+str(i) , body.text)
        else:
            r.set('irna#'+str(i) , "None")
    else:
        r.set('irna#'+str(i) , "None")
print("--- %s seconds ---" % ( time.time() - start_time ))

82120000
82120500
82121000


In [28]:
start_time = time.time()

from hazm import *
tagger = POSTagger(model='../dataset/tagger/postagger.model')
#tagger = StanfordPOSTagger(model_filename='../dataset/persian.tagger', path_to_jar='../dataset/stanford-postagger.jar')
clean_train = []
clean_train_original = []
cnt = 0
for i in r.keys(pattern='irna#82120[0|1|2]**'):
    body = r.get(i)
    cnt += 1
    if cnt % 50 == 0:
        print(cnt)
    if body == 'None':
        continue
    body = body.decode('utf-8')
#    print(body)
    norm = Normalizer()
    lemmatizer = Lemmatizer()
    body = norm.normalize(body)
    words = word_tokenize(body)
    words = [word for word in words if word not in list(swfa[0]) ]
    for idx , j in enumerate(words):
#        words[idx] = stemmer.stem(j)
        k = tagger.tag([words[idx]])
#        print(k)
        if k[0][1] in ["PUNC","NUM","V","ADV","AJ"] :
            words[idx] = ''
            continue
        words[idx] = lemmatizer.lemmatize(words[idx] , pos =k[0][1])

        #print(tagger.tag(j))
    clean_train_original.append(words)
    clean_train.append( " ".join( words ) )
#    print(clean_train)

print("--- %s seconds ---" % ( time.time() - start_time ))

50
100
150
200
250
300
--- 63.86656975746155 seconds ---


In [29]:
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 1000)
train_data_features_original = vectorizer.fit_transform(clean_train)
train_data_features = train_data_features_original.toarray()
vocab = vectorizer.get_feature_names()
print(len(vocab))
dist = np.sum(train_data_features, axis=0)

1000


In [30]:
model = lda.LDA(n_topics = 50 , n_iter=300, random_state=1)
model.fit(train_data_features)

In [31]:
topic_word = model.topic_word_  # model.components_ also works
n_top_words = 10
for i, topic_dist in enumerate(topic_word):
    np.array(vocab)[np.argsort(topic_dist)]
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-n_top_words:-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: گروه گزارش شبک نفر دست خبرگزاری عنوان ماه وارد
Topic 1: سفر کشور وزیر ماه خراسان جمهوری امضا سازمان جنوب
Topic 2: استان البرز کرج جاده شهرسازی چالوس شورا طالقان امنیت
Topic 3: استان بخش توجه کار فراهم ایجاد مجموعه فعالیت انجام
Topic 4: کشور اشاره کالا سال رفاه حوزه مقابله رشد مالی
Topic 5: سرمایه استان واحد استاندار پروژه تولید گزارش ستاد اشتغال
Topic 6: استان نفر ایرنا بیان خبرنگار اعتبار عنوان ساخت آیین
Topic 7: تیم خونه فوتبال لیگ فدراسیون بازیکن باشگاه جوان سرمربی
Topic 8: رئیس جمهوری شیراز احیا جلسه معاون ایرنا مراکز قرار
Topic 9: دانشگاه انقلاب شهدا شهید قم استاد مدافع شهادت دفاع
Topic 10: استان ثبت کرمان اسناد گذشته اداره املاک مالکیت مدیرکل
Topic 11: حزب وزیر رهبر انتخابات لبنان دولت مجلس تصمیم اعلام
Topic 12: کودک شهرستان همکار مساجد برنامه مادر زن کانون توزیع
Topic 13: دستگاه خبر اطلاع دادگستری انتشار محل پرونده رسان موضوع
Topic 14: شهرستان کیلومتر شامل خرید گندم واقع هکتار ریال دست
Topic 15: داعش سوریه عراق گروه نیرو پایگاه عملیات ها خبر
Topic 16: مازندران منطقه سرم

In [32]:
from gensim import corpora, models , similarities

In [59]:
dictionary = corpora.Dictionary(clean_train_original)
corpus = [dictionary.doc2bow(text) for text in clean_train_original]
tfidf = models.TfidfModel(corpus)

In [60]:
corpus_tfidf = tfidf[corpus]
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=10) # initialize an LSI transformation
corpus_lsi = lsi[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

In [61]:
for i in range(30):
    print(lsi.print_topic(i))
    print ("----------")

1.000*"None" + 0.000*"اسپانیا" + -0.000*"رایانه" + -0.000*"ابررایانه" + -0.000*"استخر" + -0.000*"آمریکا" + -0.000*"آلمان" + -0.000*"چین" + -0.000*"ناتو" + 0.000*"وزارت"
----------
0.851*"" + 0.083*"اقتصاد" + 0.069*"مقاومت" + 0.068*"گردشگری" + 0.067*"طرح" + 0.066*"استان" + 0.065*"کشور" + 0.056*"شهر" + 0.055*"شهرستان" + 0.053*"خونه"
----------
0.707*"خونه" + 0.319*"تیم" + 0.311*"لیگ" + 0.213*"مازندران" + 0.199*"بازیکن" + 0.126*"فوتبال" + 0.115*"باشگاه" + 0.101*"سرمربی" + 0.089*"مرزبان" + 0.072*"کریم"
----------
-0.488*"بحرین" + -0.229*"عیسی" + 0.192*"اقتصاد" + -0.186*"شیخ" + 0.177*"گردشگری" + 0.172*"مقاومت" + -0.143*"قاسم" + 0.111*"تعاون" + -0.100*"حزب" + 0.099*"بنا"
----------
-0.295*"اقتصاد" + 0.290*"گردشگری" + -0.274*"مقاومت" + 0.241*"بنا" + 0.235*"شیراز" + -0.202*"تعاون" + 0.201*"احیا" + 0.198*"میراث" + 0.195*"مرمت" + 0.158*"بافت"
----------
0.416*"بحرین" + 0.248*"اقتصاد" + 0.220*"مقاومت" + 0.197*"عیسی" + 0.167*"تعاون" + 0.160*"شیخ" + 0.144*"قاسم" + 0.143*"گردشگری" + 0.110*"تعاونی" +

In [69]:
>>> hdp = models.HdpModel(corpus, dictionary.id2token)
>>> hdp.print_topics(topics=4, topn=10)

['topic 0: 0.090* + 0.004*اقتصاد + 0.004*اورژانس + 0.003*اصفهان + 0.003*مقاومت + 0.003*سال + 0.002*پایگاه + 0.002*ترامپ + 0.002*نظر + 0.002*بیان',
 'topic 1: 0.055* + 0.006*the + 0.003*and + 0.003*to + 0.003*that + 0.002*of + 0.002*in + 0.002*قانون + 0.002*ایران + 0.002*آمریکا',
 'topic 2: 0.055* + 0.003*بحرین + 0.002*مواد + 0.002*حزب + 0.002*ورزش + 0.002*بیان + 0.002*پیشگیر + 0.002*استان + 0.002*بخش + 0.001*تعاون',
 'topic 3: 0.060* + 0.004*سال + 0.003*استان + 0.003*اسناد + 0.002*ثبت + 0.002*کرمان + 0.002*فوتبال + 0.002*طرح + 0.002*گردشگری + 0.002*املاک']

<script src="https://d3js.org/d3.v4.min.js"></script>


%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min'
  }
});
element.append("<div id='chart1'></div>");